In [74]:
import cv2
import numpy as np
import PIL.Image
from io import BytesIO
import IPython.display
import ipywidgets as widgets

from IPython.display import Javascript
def preventScrolling():
    disable_js = """
    IPython.OutputArea.prototype._should_scroll = function(lines) {
        return false;
    }
    """
    display(Javascript(disable_js))

preventScrolling()

def images_to_widgets(imgs, color=True):
    widgets = []
    for img in imgs:
        widgets.append(imwidget(np.array(img), color=color))
    return widgets

def apply_threshold(img, plane, low, high):
    image_width = img.shape[0]
    image_height = img.shape[1]
    thresh_img = np.full((image_height, image_width), 255, dtype=np.uint8)         
    _,low_img = cv2.threshold(np.asarray(plane, np.float32), int(low), 255, cv2.THRESH_BINARY)            
    _,high_img = cv2.threshold(np.asarray(plane, np.float32), int(high), int(255), cv2.THRESH_BINARY_INV)            
    thresh_band_img = cv2.bitwise_and(low_img, high_img)
    return thresh_band_img

def apply_morphography(img):
    kernelClose = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
    kernelOpen = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    filtered_img_close = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernelClose)
    filtered_img_open = cv2.morphologyEx(filtered_img_close, cv2.MORPH_OPEN, kernelOpen)
    return filtered_img_open

def on_value_change(index, change, planes, imgs):
    low_thresholds[index] = change[0]
    high_thresholds[index] = change[1]
    thresh_imgs = []
    for i, plane in enumerate(hue_planes):
        stop_img = cv2.imread(stop_files[i])
        thresh_img = apply_threshold(stop_img, plane, low_thresholds[index], high_thresholds[index])
    imgs = thresh_imgs
    widgets = images_to_widgets(imgs, color=False)
    widgets.append(ranges[index])
    displays[index] = widgets
    on_value_change_final()
    
def on_value_change_hue(change):
    on_value_change(index=0, change=change["new"], planes=hue_planes, imgs=hue_imgs)

def on_value_change_saturation(change):
    on_value_change(index=1, change=change["new"], planes=saturation_planes, imgs=saturation_imgs)
    
def on_value_change_grey(change):
    on_value_change(index=2, change=change["new"], planes=grey_planes, imgs=grey_imgs)

def on_value_change_final():
    final_imgs = []
    for i, img in enumerate(stop_files):
        img = cv2.imread(img)
        image_width = img.shape[0]
        image_height = img.shape[1]
        thresh_img = np.full((image_height, image_width), 255, dtype=np.float32)         
        thresh_img = cv2.bitwise_and(thresh_img, grey_imgs[i])
        thresh_img = cv2.bitwise_and(thresh_img, hue_imgs[i])
        thresh_img = cv2.bitwise_and(thresh_img, saturation_imgs[i])
        final_imgs.append(thresh_img)
    results = final_imgs
    displays[3] = images_to_widgets(results, color=False)
    displays[4] = widgets.VBox([widgets.HBox([displays[0], displays[1], displays[2]]), displays[3]])
    display(displays[4])
    
def imwidget(img, fmt='jpeg',width=300, color=True):
    if(color):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    f = BytesIO()
    new_p = PIL.Image.fromarray(img)
    if new_p.mode != 'RGB':
        new_p = new_p.convert('RGB')
    new_p.save(f, fmt)
    return widgets.Image(value=f.getvalue(), format=fmt, width=width)


def createRange(low,high,name):
    rangeH = np.linspace(low,high, num=high+1, dtype=int)
    options = [(f'''{i}\t''', f'''\t{i}''') for i in rangeH]
    return widgets.SelectionRangeSlider(
        options=options,
        index=(low, high),
        description=name,
        disabled=False,
        layout=widgets.Layout(width="300px"),
        style={'description_width': 'initial'}
    )

hue_range = createRange(0,255,"Hue")
hue_range.observe(on_value_change_hue, names='value')
hue_planes = []
hue_imgs = []

saturation_range = createRange(0,255,"Saturation")
saturation_range.observe(on_value_change_saturation, names='value')
saturation_planes = []
saturation_imgs = []

grey_range = createRange(0,255,"Grey")
grey_range.observe(on_value_change_grey, names='value')
grey_planes = []
grey_imgs = []



low_thresholds = [134, 70, 50]
high_thresholds = [255, 255, 255]
ranges = [hue_range, saturation_range, grey_range]
stop_files = ["stop0.jpg", "stop1.jpg", "stop2.jpg", "stop3.jpg", "stop4.jpg"]
stops = []
results = []



for stop in stop_files:
    
    stop = cv2.imread(stop)
    
    hue_imgs.append(stop)
    saturation_imgs.append(stop)
    grey_imgs.append(stop)
    stop_img = imwidget(np.array(stop))
    stops.append(stop_img)
    
    hsv_img = cv2.cvtColor(stop, cv2.COLOR_BGR2HSV)
    hue_plane, saturation_plane, grey_plane = cv2.split(hsv_img)
    hue_planes.append(hue_plane)
    saturation_planes.append(saturation_plane)
    grey_planes.append(grey_plane)


hueBox = images_to_widgets(hue_imgs)
saturationBox = images_to_widgets(saturation_imgs)
greyBox = images_to_widgets(grey_imgs)
results = stops.copy()


hueBox.append(ranges[0])
saturationBox.append(ranges[1])
greyBox.append(ranges[2])
resultsBox = results


hueBox = widgets.VBox(hueBox)
greyBox = widgets.VBox(greyBox)
saturationBox = widgets.VBox(saturationBox)
resultsBox = widgets.HBox(resultsBox, layout=widgets.Layout(width="100%", display='inline-flex',flex_flow='row wrap', flex_grow="2", align_item="center", justify_content="space-around"))

displays = [hueBox, saturationBox, greyBox, resultsBox]
displays.append(widgets.VBox([widgets.HBox([displays[0], displays[1], displays[2]]), displays[3]]))
display(displays[4])

<IPython.core.display.Javascript object>

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-52oirelq\opencv\modules\core\src\arithm.cpp:234: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and type), nor 'array op scalar', nor 'scalar op array' in function 'cv::binary_op'


error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-52oirelq\opencv\modules\core\src\arithm.cpp:234: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and type), nor 'array op scalar', nor 'scalar op array' in function 'cv::binary_op'


error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-52oirelq\opencv\modules\core\src\arithm.cpp:234: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and type), nor 'array op scalar', nor 'scalar op array' in function 'cv::binary_op'


error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-52oirelq\opencv\modules\core\src\arithm.cpp:234: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and type), nor 'array op scalar', nor 'scalar op array' in function 'cv::binary_op'


error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-52oirelq\opencv\modules\core\src\arithm.cpp:234: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and type), nor 'array op scalar', nor 'scalar op array' in function 'cv::binary_op'


error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-52oirelq\opencv\modules\core\src\arithm.cpp:234: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and type), nor 'array op scalar', nor 'scalar op array' in function 'cv::binary_op'


In [12]:
import cv2
import time
import PIL.Image
from io import BytesIO
import IPython.display
import numpy as np

#Use 'jpeg' instead of 'png' (~5 times faster)
def array_to_image(a, fmt='jpeg'):
    #Create binary stream object
    f = BytesIO()
    #Convert array to binary stream object
    PIL.Image.fromarray(a).save(f, fmt)
    return IPython.display.Image(data=f.getvalue())

def nothing(x):pass



# binary_img = cv2.adaptiveThreshold(src=gray_img,maxValue=255,  # output value where condition met
#                                    adaptiveMethod=cv2.ADAPTIVE_THRESH_MEAN_C,
#                                    thresholdType=cv2.THRESH_BINARY,  # threshold_type
#                                    blockSize=51,  # neighborhood size (a large odd number)
#                                    C=-10)  # a constant to subtract from mean

d1 = IPython.display.display("", display_id=1)
d2 = IPython.display.display("", display_id=2)
d3 = IPython.display.display("", display_id=3)
low_thresholds = [134, 70, 50]
high_thresholds = [255, 255, 255]

stop0 = cv2.imread("stop0.jpg")
stop1 = cv2.imread("stop1.jpg")
stop2 = cv2.imread("stop2.jpg")
stop3 = cv2.imread("stop3.jpg")
stop4 = cv2.imread("stop4.jpg")

stops = [stop0,stop1,stop2,stop3,stop4]
bgr_img = stop4
image_height = bgr_img.shape[0]    
image_width = bgr_img.shape[1]# Convert BGR to HSV.
hsv_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2HSV)# Split into the different bands.
planes = cv2.split(hsv_img)    
windowNames = ["Hue image", "Saturation image", "Gray image"]
try:
#     for i in range(3):        
#         cv2.namedWindow(windowNames[i])# Create trackbars.
#     for i in range(3):        
#         cv2.createTrackbar("Low", windowNames[i], low_thresholds[i], 255, nothing)        
#         cv2.createTrackbar("High", windowNames[i], high_thresholds[i], 255, nothing)
    while True:# Create output thresholded image.
        thresh_img = np.full((image_height, image_width), 255, dtype=np.uint8)
        for i in range(3):            
#             low_val = cv2.getTrackbarPos("Low", windowNames[i])            
#             high_val = cv2.getTrackbarPos("High", windowNames[i]) 
            low_val = low_thresholds[i]
            high_val = high_thresholds[i]
            _,low_img = cv2.threshold(planes[i], low_val, 255, cv2.THRESH_BINARY)            
            _,high_img = cv2.threshold(planes[i], high_val, 255, cv2.THRESH_BINARY_INV)            
            thresh_band_img = cv2.bitwise_and(low_img, high_img)            
            #cv2.imshow(windowNames[i], thresh_band_img)# AND with output thresholded image.
            thresh_img = cv2.bitwise_and(thresh_img, thresh_band_img)        


        kernelClose = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
        kernelOpen = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        filtered_img_close = cv2.morphologyEx(thresh_img, cv2.MORPH_CLOSE, kernelClose)
        filtered_img_open = cv2.morphologyEx(filtered_img_close, cv2.MORPH_OPEN, kernelOpen)
        d1.update(array_to_image(filtered_img_close))
        d2.update(array_to_image(filtered_img_open))
        cv2.imwrite("stop_output4.jpg", filtered_img_open)

except:
    pass
finally:
    cv2.destroyAllWindows()
    IPython.display.clear_output()
    



<div style="display:flex; flex-direction:row; justify-contents:space-around;font-size:3rem;width: 100%;color:rgba(255,255,255,1);padding:2rem 5rem;">
    <div style="background-color: rgba(0,0,0,0.3);padding:1rem;">
        <h2>Hue</h2>
        <h4 style="margin-bottom: 1rem;">Low</h4>
        134
        <h4 style="margin-bottom: 1rem;">High</h4>
        255
    </div>
    <div style="background-color: rgba(0,0,0,0.6);padding:1rem;">
       <h2>Saturation</h2>
        <h4 style="margin-bottom: 1rem;">Low</h4>
        50
        <h4 style="margin-bottom: 1rem;">High</h4>
        255
    </div>
    <div style="background-color: rgba(0,0,0,0.9);padding:1rem;">
        <h2>Grey</h2>
        <h4 style="margin-bottom: 1rem;">Low</h4>
        70
        <h4 style="margin-bottom: 1rem;">High</h4>
        255
    </div>
</div>

<div style="display:flex; flex-direction:row; justify-contents:space-around;font-size:3rem;width: 100%;color:rgba(0,0,0,1);padding:2rem 5rem;">
    <div style="margin:1rem">
        <h4>Stop0</h4>
        <img src="stop_output0.jpg" width="200px">
    </div>
       <div style="margin:1rem">
        <h4>Stop1</h4>
        <img src="stop_output1.jpg" width="200px">
    </div>
        <div style="margin:1rem">
        <h4>Stop2</h4>
        <img src="stop_output2.jpg" width="200px">
    </div>
        <div style="margin:1rem">
        <h4>Stop3</h4>
        <img src="stop_output3.jpg" width="200px">
    </div>
        <div style="margin:1rem">
        <h4>Stop4</h4>
        <img src="stop_output4.jpg" width="200px">
    </div>
</div>
